In [11]:
import cv2
import numpy as np


In [12]:
import time
from threading import Thread
VideoName = "saotrailer.mp4"
Compress = "comressed_video1.npy"
BS = 8 #розмір макроблоку
Step = 4 #залежить пошукова область(2*Step-1)
S2=1
sum_of_err=0


def findbias(first, second):
    bias = np.zeros((first.shape[0]//BS, first.shape[1]//BS, 2))
    onethread(bias, 0, first.shape[0]//BS, first, second, first.shape[1]//BS, sum_of_err)
    return bias
def onethread(bias, fro, to, first, second, width, sum_of_err):
    for i in range(fro, to):
        for j in range(width):
            S=Step
            imin = 0
            jmin = 0
            while(S > 0):
                im, jm, norm= findmindif(first, second, S, i*BS+imin, j*BS+jmin)
                imin += im
                jmin += jm
                S = S // 2
            sum_of_err+=norm
            bias[i, j, 0] = imin
            bias[i, j, 1] = jmin

def findmindif(first, second, S, x, y):
    block = first[x:x+BS , y:y+BS]
    mind = mad(block, second[x:x+BS , y:y+BS])
    imin, jmin = 0,0
    positions  =  [(i,j) for i in range(-S,S+1,8) for j in range(-S,S+1,8) if (i,j)!=(0,0)]
#    positions +=  [(i,j) for i in range(-S2,S2+1) for j in range(-S2,S2+1) if (i,j)!=(0,0)]
    for pos in positions:
        i,j=pos
        if x + i >= 0 and x + i + BS <= second.shape[0] \
                and y + j >= 0 and y + j + BS <= second.shape[1]:
            norm = mad(block, second[x+i:x+i+BS,y+j:y+j+BS])
            if mind > norm:
                mind = norm
                imin,jmin = i,j
    return imin, jmin, mind

def mad(a, b):
    return 1/BS**2*np.sum(np.sum(np.abs(a - b), axis=1),axis=0)

In [13]:
#Encoder

cap = cv2.VideoCapture(VideoName)
file = open(Compress , 'wb')

start_time = time.time()
while(1):
    ret, frame1 = cap.read()
    shape = frame1.shape
    if ret == True:
        ret, frame2 = cap.read()
        if ret == True:
            np.save(file, frame2)
            first = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
            second = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
            bias = findbias(second, first)
            np.save(file, bias.astype(np.int8))
            ret, frame3 = cap.read()
            if ret == True:
                third = cv2.cvtColor(frame3,cv2.COLOR_BGR2GRAY)
                bias = findbias(second, third)
                np.save(file, bias.astype(np.int8))
            else:
                break
        else:
            np.save(file, frame1)
            break
    else:
        break


    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

print(f"time of encoding: {round(time.time() - start_time, 4)}")

file.close()
cap.release()
cv2.destroyAllWindows()

time of encoding: 597.5797


In [14]:
#Decoder
file = open(Compress, 'rb')
out = cv2.VideoWriter('resultvideo.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame1.shape[1],frame1.shape[0]))

start_time = time.time()
while(1):
    try:
        frame = np.load(file)
        try:
            bias = np.load(file)
            newframe = frame.copy()
            for i in range(bias.shape[0]):
                for j in range(bias.shape[1]):
                    block = frame[i*BS:(i+1)*BS,j*BS:(j+1)*BS,:]
                    newframe[int(i*BS+bias[i][j][0]):int((i+1)*BS+bias[i][j][0]),int(j*BS+bias[i][j][1]):int((j+1)*BS+bias[i][j][1]),:] = block
            cv2.imshow('video',newframe)
            cv2.imshow('video',frame)
            out.write(newframe)
            out.write(frame)
            try:
                bias = np.load(file)
                newframe = frame.copy()
                for i in range(bias.shape[0]):
                    for j in range(bias.shape[1]):
                        block = frame[i*BS:(i+1)*BS,j*BS:(j+1)*BS,:]
                        newframe[int(i*BS+bias[i][j][0]):int((i+1)*BS+bias[i][j][0]),int(j*BS+bias[i][j][1]):int((j+1)*BS+bias[i][j][1]),:] = block
                cv2.imshow('video',newframe)
                out.write(newframe)
            except ValueError:
                break
        except ValueError:
            cv2.imshow('video',frame)
            out.write(frame)
            break
    except ValueError:
        break

    #cv2.waitKey(0)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

print(f"time of decoding: {round(time.time() - start_time, 4)}")

file.close()
out.release()
cap.release()
cv2.destroyAllWindows()

print("Sum or error: "+sum_of_err)

time of decoding: 64.7088


TypeError: can only concatenate str (not "int") to str